In [1]:
from pathlib import Path

from twitter_trends import ArgsTwitterTrends, ParamsTwitterTrends, WOEIDCountry, requests_and_process
from scraping_kit import BotScraper, load_db_and_bots
from scraping_kit.db.models.search import Search


path_data = Path("data")
db_tw, bots = load_db_and_bots(path_data, "scrape_tw")

/home/aprieto/.personal/scraping_kit/env/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-01-13 13:27:18.180509: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-01-13 13:27:18.216193: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-13 13:27:18.216228: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-13 13:27:18.217493: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory f

Collection Names: ['trends', 'raw', 'search']
Bots: bots=[BotScraper(acc_name='Vera')]


In [ ]:
from scraping_kit.db.models.topics import Topic
from concurrent.futures import ThreadPoolExecutor, as_completed
max_workers = 2
n_first_texts = 10

topics = []
with ThreadPoolExecutor(max_workers=max_workers) as pool:
    iter_futures = (pool.submit(Topic.create_from_search, Search(**search_doc), classifier, topics_1, n_first_texts)
                    for search_doc in searchs_docs)#db_tw.coll.search.find())

    for i, future in enumerate(as_completed(iter_futures)):
        topic = future.result()
        topics.append(topic)

#search = Search(**db_tw.coll.search.find_one())
#topic = Topic.create_from_search(search, classifier, topics_1, n_first_texts=10)
#topic

In [ ]:
topics[3]

In [ ]:
for i, search_doc in enumerate(db_tw.coll.search.find()):
    search = Search(**search_doc)
    #texts = search.get_texts()
    #len_texts.append(len(texts))

In [ ]:
from scraping_kit.db.models.topics import Topic

topics: list[Topic] = []
for doc in db_tw.coll.topics.find():
    topic = Topic(**doc)
    topics.append(topic)


In [ ]:
import json
with open("CLASSES_TWITTER.json", "r") as f:
    CLASSES_TWITTER = json.load(f)

In [ ]:
from transformers import pipeline
classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")



In [ ]:
topics[0]

In [ ]:
threshold = 0.5
text = topics[0].topics_1.text
labels, scores = topics[0].get_labels_scores_thresh(threshold)
labels

In [ ]:
print(len(labels))
TOPICS2 = []
for label in labels:
    TOPICS2.extend(CLASSES_TWITTER[label])
TOPICS2

In [ ]:
asd = classifier(
    text,
    candidate_labels = TOPICS2,
    multi_label = True
)
asd

In [ ]:
import json
with open("asd.json", "w") as f:
    json.dump(topics, f, default=lambda obj: obj.isoformat())